In [3]:
# list datasets for yack
from rouskinhf import *
import pandas as pd

datasets = ["zuber", 'bpRNA','RNAstralign','archiveII',"ribo500-blast"]

data = pd.DataFrame()
for name in datasets:
    ds = pd.DataFrame.from_dict(get_dataset(name), orient='index')
    data = pd.concat([data, ds], axis=0)  
      
data.reset_index(inplace=True)
data.rename(columns={'index': 'reference'}, inplace=True)

zuber: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zuber: Download complete. File saved at data/zuber/data.json
bpRNA: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

bpRNA: Download complete. File saved at data/bpRNA/data.json
RNAstralign: Downloading dataset from HuggingFace Hub...


...

RNAstralign: Download complete. File saved at data/RNAstralign/data.json
archiveII: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

archiveII: Download complete. File saved at data/archiveII/data.json
ribo500-blast: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ribo500-blast: Download complete. File saved at data/ribo500-blast/data.json


In [4]:
# rename non-unique references
for i, group in data.groupby('reference'):
    if len(group) > 1:
        for j, row in group.iterrows():
            data.loc[j, 'reference'] = row['reference'] + '__' + str(j)

assert len(data['reference'].unique()) == len(data)

data.set_index('reference', inplace=True)
dump_json(data.to_dict(orient='index'), 'datasets.json')

In [6]:
dump_json(
    convert('json', 'datasets.json', name='yack'),
    path="yack.json")

Parsing json file: 100%|██████████| 144650/144650 [00:04<00:00, 31869.91it/s]


Over a total of 144650 datapoints, there are:
### OUTPUT
- ALL: 125671 valid datapoints
- INCLUDED: 0 duplicate sequences with different structure / dms / shape
### MODIFIED
- 0 multiple sequences with the same reference (renamed reference)
### FILTERED OUT
- 0 invalid datapoints (ex: sequence with non-regular characters)
- 0 datapoints with bad structures
- 18979 duplicate sequences with the same structure / dms / shape
- 0 datapoints removed because of low AUROC (<0.8)


In [8]:
upload_dataset('yack.json', exist_ok=True)

yack.json:   0%|          | 0.00/262M [00:00<?, ?B/s]

## Yack train / valid

In [3]:
from rouskinhf import *

# VALID: 
# pick 4096 random references from the dataset
import numpy as np
data = get_dataset('yack')
valid_refs = np.random.choice(list(data.keys()), size=4096, replace=False)

# save valid dataset
data_valid = {k: data[k] for k in valid_refs}
dump_json(data_valid, 'yack_valid.json')
upload_dataset('yack_valid.json', exist_ok=True)

# TRAIN:
# keep only the references that are not in the valid dataset
data_train = {k: data[k] for k in data if k not in valid_refs}
dump_json(data_train, 'yack_train.json')
upload_dataset('yack_train.json', exist_ok=True)

yack_train.json:   0%|          | 0.00/253M [00:00<?, ?B/s]